# Optimizing ONNX

In [1]:
!pip install onnxruntime onnx transformers optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.3/252.3 kB 24.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.0 MB

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import torch
from torch import optim
import torch.nn.functional as F

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer

from pathlib import Path
import timeit
import onnxruntime as ort
from onnxruntime import InferenceSession
from onnxruntime.transformers.optimizer import optimize_model
from optimum.onnxruntime import ORTModelForSequenceClassification

The only difference is that we use the `optimize_model` function to optimize the graph inside the model.

In [9]:
# Create a PATH to save the model
model_onnx_path = Path("/content/drive/MyDrive/Models/indobert-onnx")
optimized_onnx_path = str(model_onnx_path / "optimized.onnx")

# Optimize the model using ORT transformer optimizer with its default values
optimized_model = optimize_model(
    input=str(model_onnx_path / "model.onnx"),
    model_type="bert",
    use_gpu=False
)

# Save the model
optimized_model.save_model_to_file(optimized_onnx_path)

In [10]:
token_load_path = Path("/content/drive/MyDrive/Models/indobert")
tokenizer = BertTokenizer.from_pretrained(token_load_path)

# Create inputs for the model
text = ['Bahagia hatiku melihat pernikahan putri sulungku yang cantik jelita']
inputs = tokenizer(text)

inputs_onnx = dict(
    input_ids=np.array(inputs["input_ids"]).astype("int64"),
    attention_mask=np.array(inputs["attention_mask"]).astype("int64"),
    token_type_ids=np.array(inputs["token_type_ids"]).astype("int64")
)

inputs_onnx

{'input_ids': array([[    2,  4771, 10413,   722,  3300,  3466, 19227,   457,    34,
          2176, 17377,   155,     3]]),
 'attention_mask': array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 'token_type_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [11]:
# Create ONNX session to do inference
opt_sess = InferenceSession(optimized_onnx_path, providers=["CPUExecutionProvider"])

In [12]:
i2w = {0: 'positive', 1: 'neutral', 2: 'negative'}

# Do the inference
logits = opt_sess.run(None, input_feed=inputs_onnx)[0]
label = torch.topk(torch.from_numpy(logits), k=1, dim=-1)[1].squeeze().item()
probability = F.softmax(torch.from_numpy(logits), dim=-1).squeeze()[label].item()

print(f"Label: {i2w[label]}\nProbability: {probability}")

Label: positive
Probability: 0.9998039603233337


The optimized model produces the same result as the non-optimized one.